<a href="https://colab.research.google.com/github/llgelarall/CUDAProgrammingAsATA/blob/main/How%20to%20install%20CUDA%2BOpenCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. CUDA Installation in Colab:

In [ ]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter
!nvidia-smi
!nvcc --version

A sample cuda code

In [ ]:
%%cuda
#include <stdio.h>
__global__ void hello(){
 printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}
int main(){
 hello<<<2, 2>>>();
 cudaDeviceSynchronize();
}


# 2.**OpenCV installation in Colab for CUDA programming**
---



**2.1 Requirements Installation**

---




In [ ]:
!apt update && apt install -y cmake g++ wget unzip
!apt install -y libgtk2.0-dev

2.2 By executing the commands OpenCV will be downloaded

---



In [ ]:
!wget -O opencv.zip https://github.com/opencv/opencv/archive/4.5.2.zip
!wget -O opencv_contrib.zip https://github.com/opencv/opencv_contrib/archive/4.5.2.zip
!unzip opencv.zip
!unzip opencv_contrib.zip

2.3 cmake is going to build the opencv in customized folder

---



In [ ]:
!mkdir /content/opencv-4.5.2/build
%cd /content/opencv-4.5.2

!cmake -D CMAKE_BUILD_TYPE=RELEASE \-D CMAKE_INSTALL_PREFIX=/usr/local \-D INSTALL_PYTHON_EXAMPLES=ON \-D INSTALL_C_EXAMPLES=OFF \-D OPENCV_ENABLE_NONFREE=ON \-D WITH_CUDA=ON \-D WITH_CUDNN=ON \-D OPENCV_DNN_CUDA=ON \-D ENABLE_FAST_MATH=1 \-D CUDA_FAST_MATH=1 \-D CUDA_ARCH_BIN=7.5

!make -j8
!make install


**2.4 to test the openCd installation**

---



In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/cv2/python-3.6')

import cv2
print(cv2.__version__)


4.8.0


**2.5 create a new cuda file (*.cu), For instance, "main.cu" in the main root (/content)and then copy your cuda code.**


---



In [ ]:
%%cuda
#include <opencv2/opencv.hpp>
#include <opencv2/core/cuda.hpp>
#include <iostream>

using namespace std;
using namespace cv;
using namespace cv::cuda;

__global__ void rgb2grayKernel(uchar3 *input, uchar *output, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    if (x < width && y < height) {
        uchar3 rgb = input[y * width + x];
        output[y * width + x] = (uchar)(0.299f * rgb.x + 0.587f * rgb.y + 0.114f * rgb.z);
    }
}

void rgb2gray(const Mat& input, Mat& output) {
    int width = input.cols;
    int height = input.rows;

    // Allocate device memory
    uchar3 *d_input;
    uchar *d_output;
    cudaMalloc(&d_input, sizeof(uchar3) * width * height);
    cudaMalloc(&d_output, sizeof(uchar) * width * height);

    // Copy data from OpenCV input image to device memory
    cudaMemcpy(d_input, input.ptr(), sizeof(uchar3) * width * height, cudaMemcpyHostToDevice);

    // Kernel configuration
    dim3 blockSize(16, 16);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // Launch the kernel
    rgb2grayKernel<<<gridSize, blockSize>>>(d_input, d_output, width, height);

    // Copy the processed data back to the output image
    cudaMemcpy(output.ptr(), d_output, sizeof(uchar) * width * height, cudaMemcpyDeviceToHost);

    // Free device memory
    cudaFree(d_input);
    cudaFree(d_output);
}

int main() {
    try {
        // Read the image file
        Mat h_img = imread("input.jpg");
        if (h_img.empty()) {
            cerr << "Error loading image file." << endl;
            return 1;
        }

        // Convert the image from BGR to RGB (OpenCV uses BGR by default)
        Mat h_img_rgb;
        cvtColor(h_img, h_img_rgb, COLOR_BGR2RGB);

        // Create an output image (grayscale)
        Mat h_img_gray(h_img.rows, h_img.cols, CV_8UC1);

        // Convert RGB to Grayscale using CUDA
        rgb2gray(h_img_rgb, h_img_gray);

        // Show the result
        imshow("Original Image", h_img);
        imshow("Grayscale Image", h_img_gray);
        waitKey(0);
    } catch (const exception& e) {
        cerr << "Error: " << e.what() << endl;
        return 1;
    }

    return 0;
}


**2.6 to ensure the OpenCV is successfully installed **

---



In [ ]:
!pkg-config --modversion opencv4

**2.7 (optional) if you are not in the main root (/content) run these commands**

---



In [ ]:
%cd ..
!ls

**2.8 Compile and Execute your code**

---



In [ ]:
!nvcc main.cu -o rgb2gray_app `pkg-config --cflags --libs opencv4`

**2.9 Run the executable created**

---



In [ ]:
!./rgb2gray_app